In [2]:
import cv2
import numpy as np

In [3]:
# implementing required functions 
#for faster computation we only get 1 frame in every 2 frames
def save_frames(videoAddress,number_of_frames,name):
    frames= []
    vidcap = cv2.VideoCapture(videoAddress)
    success,image = vidcap.read()
    count = 0
    while success and count<number_of_frames:
        path="frame_"+str(name)+str(count)+".jpg"
        cv2.imwrite(path, image)     # save frame as JPEG file      
        frames.append(image)
        vidcap.read()
        success,image = vidcap.read()
        # print('Read a new frame: ', success)
        count += 1
    print("Captured "+str(count) + " frames .")
    return frames


In [4]:
def get_feature_points(imgAddress):
    img=cv2.imread(imgAddress)
    img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Create ORB detector with 5000 features.
    orb_detector = cv2.ORB_create(5000)
 
# Find keypoints and descriptors.
# The first arg is the image, second arg is the mask
#  (which is not required in this case).
    key_points, descriptors= orb_detector.detectAndCompute(img_gray, None)
    return key_points,descriptors,img
    

In [5]:
def match(src_descriptors,train_descriptors):
    # Match features between the two images.
# We create a Brute Force matcher with
# Hamming distance as measurement mode.
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck = True)
 
# Match the two sets of descriptors.
    matches = matcher.match(src_descriptors, train_descriptors)
    # Sort matches on the basis of their Hamming distance.
    matches.sort(key = lambda x: x.distance)
# Take the top 90 % matches forward.
    matches = matches[:int(len(matches)*0.9)]
    no_of_matches = len(matches)
    return matches,no_of_matches


In [6]:
def homography_transform(no_of_matches,matches,src_kp,dest_kp):
    # Define empty matrices of shape no_of_matches * 2.
    src_pts = np.zeros((no_of_matches, 2))
    dest_pts = np.zeros((no_of_matches, 2))
 
    for i in range(len(matches)):
        src_pts[i, :] = src_kp[matches[i].queryIdx].pt
        dest_pts[i, :] = dest_kp[matches[i].trainIdx].pt
 
# Find the homography matrix.
    homography, mask = cv2.findHomography(src_pts, dest_pts, cv2.RANSAC)
    
    return homography, mask 

In [6]:
#steps to solve this problem :
#1-capture video
#2-get all frames from video 
#3-for each frame find book 
#4-put related video to the detected book
#5-put all images together and save the video


In [7]:
frames = save_frames("0.MOV",100)


Captured 100 frames .


In [8]:
print(frames)

[array([[[ 81, 145, 203],
        [ 85, 149, 207],
        [ 89, 153, 211],
        ...,
        [ 21,  25,  27],
        [ 18,  22,  24],
        [ 15,  19,  21]],

       [[ 75, 139, 197],
        [ 83, 147, 205],
        [ 91, 155, 213],
        ...,
        [ 22,  26,  28],
        [ 19,  23,  25],
        [ 18,  22,  24]],

       [[ 70, 136, 194],
        [ 81, 147, 205],
        [ 89, 155, 213],
        ...,
        [ 22,  26,  28],
        [ 22,  26,  28],
        [ 21,  25,  27]],

       ...,

       [[108,  91,  78],
        [108,  91,  78],
        [107,  90,  77],
        ...,
        [166, 110, 113],
        [167, 111, 114],
        [166, 110, 113]],

       [[108,  91,  78],
        [107,  90,  77],
        [107,  90,  77],
        ...,
        [164, 108, 111],
        [164, 108, 111],
        [163, 107, 110]],

       [[108,  91,  78],
        [107,  90,  77],
        [107,  90,  77],
        ...,
        [162, 106, 109],
        [162, 106, 109],
        [163, 107, 110]

In [9]:
#refrence
david_kp,david_desc,david=get_feature_points("David Copperfield.jpg")

In [10]:
frame1_kp,frame1_desc,frame1=get_feature_points("frame0.jpg")

In [11]:
matches,no_of_matches=match(david_desc,frame1_desc)

In [12]:
homography,mask=homography_transform(no_of_matches,matches,david_kp,frame1_kp)

In [48]:
transformed_img = cv2.warpPerspective(david,
                    homography, (frame1.shape[1], frame1.shape[0]))
 
# Save the output.
# cv2.imwrite('output.jpg', transformed_img)

In [16]:
cv2.imshow("sd2",frame1)
cv2.waitKey(0) 
  
#closing all open windows 
cv2.destroyAllWindows()

In [50]:
print(no_of_matches)

1026


In [7]:
def detect_and_mask(homography,mask,src_img,dest_img):
    matchesMask = mask.ravel().tolist()
    h,w,c = src_img.shape
    # print("src image shape ",src_img.shape)
    # print("dest 1 image shape ",dest_img.shape)
    pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
    dst = cv2.perspectiveTransform(pts,homography)
    dest_img = cv2.polylines(dest_img,[np.int32(dst)],True,0,3, cv2.LINE_8)
    # print("dest 2 image sha[e =",dest_img.shape)
    # print("pts =",pts)
    # print("dst=",dst)
    
    mask_zero = np.zeros(dest_img.shape, dtype=np.uint8)
    channel_count2 = dest_img.shape[2]  
    ignore_mask_color2 = (255,)*channel_count2
    corners = np.int32(dst)
    # print("in chie " , ignore_mask_color2)
    mask_empty_inside=cv2.fillConvexPoly(mask_zero, corners, ignore_mask_color2)
    # cv2.imshow("mask cover",mask_empty_inside)
    # cv2.waitKey(0) 
#closing all open windows 
    # cv2.destroyAllWindows()
    return mask_empty_inside,corners

In [49]:
mask_empty,corners=detect_and_mask(homography,mask,david,frame1)
mask_empty_not=cv2.bitwise_not(mask_empty)
frame_empty_inside=cv2.bitwise_and(frame1,mask_empty_not)
print("shape wraped" , transformed_img.shape)
print("shape frame" , frame_empty_inside.shape)


shape wraped (1080, 1920, 3)
shape frame (1080, 1920, 3)


In [53]:
out=cv2.bitwise_or(transformed_img,frame_empty_inside)
cv2.imshow("mask cover",out)
cv2.waitKey(0) 
cv2.destroyAllWindows()


In [8]:
def create_video(frames_src,frames_trailer,cover_img_file):

    count=0
    output_img=[]
    for eachframe in frames_src:
        try:
            cover_kp,cover_desc,cover=get_feature_points(cover_img_file)
            frame_kp,frame_desc,frame=get_feature_points("frame_src%d.jpg"%count)
        except():
            print("sth wrong")
            continue
        matches,no_of_matches=match(cover_desc,frame_desc)
        homography,mask=homography_transform(no_of_matches,matches,cover_kp,frame_kp)

        transformed_img = cv2.warpPerspective(frames_trailer[count],
                    homography, (frame.shape[1], frame.shape[0]))
        mask_empty,corners=detect_and_mask(homography,mask,cover,frame)
        mask_empty_not=cv2.bitwise_not(mask_empty)
        frame_empty_inside=cv2.bitwise_and(frame,mask_empty_not)
        out=cv2.bitwise_or(transformed_img,frame_empty_inside)
        output_img.append(out)
        cv2.imwrite("out%d.jpg"%count,out)
        print("image number %d created ",count)
        count+=1
    return output_img


In [9]:
frames_src=[]
frames_trailer=[]
try:
    print("no cache available ")
    frames_src=save_frames("0.MOV",100,"src")
    frames_trailer=save_frames("David Copperfield.mp4",100,"trailer")
except:
    print("cache available")


no cache available 
Captured 100 frames .
Captured 100 frames .


In [10]:
out_frames=create_video(frames_src,frames_trailer,"David Copperfield.jpg")

image number %d created  0
image number %d created  1
image number %d created  2
image number %d created  3
image number %d created  4
image number %d created  5
image number %d created  6
image number %d created  7
image number %d created  8
image number %d created  9
image number %d created  10
image number %d created  11
image number %d created  12
image number %d created  13
image number %d created  14
image number %d created  15
image number %d created  16
image number %d created  17
image number %d created  18
image number %d created  19
image number %d created  20
image number %d created  21
image number %d created  22
image number %d created  23
image number %d created  24
image number %d created  25
image number %d created  26
image number %d created  27
image number %d created  28
image number %d created  29
image number %d created  30
image number %d created  31
image number %d created  32
image number %d created  33
image number %d created  34
image number %d created  35
im

In [12]:
#.avi out put 

height,width,layers=out_frames[1].shape

video=cv2.VideoWriter('david caperfield.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15,(width,height))

for j in range(0,100):
    video.write(out_frames[j])

cv2.destroyAllWindows()
video.release()

In [9]:
# creating second test

frames_src2=[]
frames_trailer2=[]
print("no cache available ")
frames_src2=save_frames("1.MOV",100,"src")
frames_trailer2=save_frames("Angels and Demons.mp4",100,"trailer")


no cache available 
Captured 100 frames .
Captured 100 frames .


In [10]:
out_frames=create_video(frames_src2,frames_trailer2,"Angels and Demons.jpg")

image number %d created  0
image number %d created  1
image number %d created  2
image number %d created  3
image number %d created  4
image number %d created  5
image number %d created  6
image number %d created  7
image number %d created  8
image number %d created  9
image number %d created  10
image number %d created  11
image number %d created  12
image number %d created  13
image number %d created  14
image number %d created  15
image number %d created  16
image number %d created  17
image number %d created  18
image number %d created  19
image number %d created  20
image number %d created  21
image number %d created  22
image number %d created  23
image number %d created  24
image number %d created  25
image number %d created  26
image number %d created  27
image number %d created  28
image number %d created  29
image number %d created  30
image number %d created  31
image number %d created  32
image number %d created  33
image number %d created  34
image number %d created  35
im

In [11]:
height,width,layers=out_frames[1].shape

video=cv2.VideoWriter('Angels and Demons.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15,(width,height))

for j in range(0,100):
    video.write(out_frames[j])

cv2.destroyAllWindows()
video.release()